# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 14 2023 10:11AM,259254,21,9265154,"NBTY Global, Inc.",Released
1,Mar 14 2023 10:03AM,259251,18,EMERSA-9260656.1,"Emersa Waterbox, LLC",Released
2,Mar 14 2023 10:03AM,259252,10,Enova-11505,Emerginnova,Released
3,Mar 14 2023 9:55AM,259247,10,PRF-55378,Bio-PRF,Released
4,Mar 14 2023 9:55AM,259247,10,PRF-55381,Bio-PRF,Executing
5,Mar 14 2023 9:55AM,259247,10,PRF-55384,Bio-PRF,Released
6,Mar 14 2023 9:55AM,259247,10,PRF-55387,Bio-PRF,Released
7,Mar 14 2023 9:55AM,259247,10,PRF-55390,Bio-PRF,Released
8,Mar 14 2023 9:55AM,259247,10,PRF-55393,Bio-PRF,Released
9,Mar 14 2023 9:55AM,259247,10,PRF-55425,Bio-PRF,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,259249,Released,1
13,259250,Released,1
14,259251,Released,1
15,259252,Released,1
16,259254,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259249,NaN,1.0
259250,NaN,1.0
259251,NaN,1.0
259252,NaN,1.0
259254,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259195,0.0,64.0
259200,0.0,2.0
259215,0.0,10.0
259221,16.0,6.0
259222,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259195,0,64
259200,0,2
259215,0,10
259221,16,6
259222,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259195,0,64
1,259200,0,2
2,259215,0,10
3,259221,16,6
4,259222,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259195,,64
1,259200,,2
2,259215,,10
3,259221,16,6
4,259222,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 14 2023 10:11AM,259254,21,"NBTY Global, Inc."
1,Mar 14 2023 10:03AM,259251,18,"Emersa Waterbox, LLC"
2,Mar 14 2023 10:03AM,259252,10,Emerginnova
3,Mar 14 2023 9:55AM,259247,10,Bio-PRF
17,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc."
27,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc."
91,Mar 14 2023 9:43AM,259250,18,"Emersa Waterbox, LLC"
92,Mar 14 2023 9:36AM,259249,10,Hush Hush
93,Mar 14 2023 9:29AM,259248,16,Sartorius Bioprocess Solutions
94,Mar 14 2023 9:07AM,259245,10,"Digital Brands, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 14 2023 10:11AM,259254,21,"NBTY Global, Inc.",,1
1,Mar 14 2023 10:03AM,259251,18,"Emersa Waterbox, LLC",,1
2,Mar 14 2023 10:03AM,259252,10,Emerginnova,,1
3,Mar 14 2023 9:55AM,259247,10,Bio-PRF,6,8
4,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",,10
5,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc.",,64
6,Mar 14 2023 9:43AM,259250,18,"Emersa Waterbox, LLC",,1
7,Mar 14 2023 9:36AM,259249,10,Hush Hush,,1
8,Mar 14 2023 9:29AM,259248,16,Sartorius Bioprocess Solutions,,1
9,Mar 14 2023 9:07AM,259245,10,"Digital Brands, LLC",,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 14 2023 10:11AM,259254,21,"NBTY Global, Inc.",1,
1,Mar 14 2023 10:03AM,259251,18,"Emersa Waterbox, LLC",1,
2,Mar 14 2023 10:03AM,259252,10,Emerginnova,1,
3,Mar 14 2023 9:55AM,259247,10,Bio-PRF,8,6
4,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",10,
5,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc.",64,
6,Mar 14 2023 9:43AM,259250,18,"Emersa Waterbox, LLC",1,
7,Mar 14 2023 9:36AM,259249,10,Hush Hush,1,
8,Mar 14 2023 9:29AM,259248,16,Sartorius Bioprocess Solutions,1,
9,Mar 14 2023 9:07AM,259245,10,"Digital Brands, LLC",10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 14 2023 10:11AM,259254,21,"NBTY Global, Inc.",1,
1,Mar 14 2023 10:03AM,259251,18,"Emersa Waterbox, LLC",1,
2,Mar 14 2023 10:03AM,259252,10,Emerginnova,1,
3,Mar 14 2023 9:55AM,259247,10,Bio-PRF,8,6
4,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",10,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 14 2023 10:11AM,259254,21,"NBTY Global, Inc.",1.0,NaN
1,Mar 14 2023 10:03AM,259251,18,"Emersa Waterbox, LLC",1.0,NaN
2,Mar 14 2023 10:03AM,259252,10,Emerginnova,1.0,NaN
3,Mar 14 2023 9:55AM,259247,10,Bio-PRF,8.0,6.0
4,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",10.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 14 2023 10:11AM,259254,21,"NBTY Global, Inc.",1.0,0.0
1,Mar 14 2023 10:03AM,259251,18,"Emersa Waterbox, LLC",1.0,0.0
2,Mar 14 2023 10:03AM,259252,10,Emerginnova,1.0,0.0
3,Mar 14 2023 9:55AM,259247,10,Bio-PRF,8.0,6.0
4,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",10.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1555436,22.0,22.0
15,518410,74.0,0.0
16,259248,1.0,0.0
18,518501,2.0,0.0
19,518443,6.0,17.0
20,259225,1.0,0.0
21,259254,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1555436,22.0,22.0
1,15,518410,74.0,0.0
2,16,259248,1.0,0.0
3,18,518501,2.0,0.0
4,19,518443,6.0,17.0
5,20,259225,1.0,0.0
6,21,259254,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,22.0,22.0
1,15,74.0,0.0
2,16,1.0,0.0
3,18,2.0,0.0
4,19,6.0,17.0
5,20,1.0,0.0
6,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,22.0
1,15,Released,74.0
2,16,Released,1.0
3,18,Released,2.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Executing,22.0,0.0,0.0,0.0,17.0,0.0,0.0
Released,22.0,74.0,1.0,2.0,6.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Executing,22.0,0.0,0.0,0.0,17.0,0.0,0.0
1,Released,22.0,74.0,1.0,2.0,6.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Executing,22.0,0.0,0.0,0.0,17.0,0.0,0.0
1,Released,22.0,74.0,1.0,2.0,6.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()